<a href="https://colab.research.google.com/github/OlgaMatykina/Internship_VK/blob/main/SVD_LightFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Коллаборативный подход

In [1]:
pip install git+https://github.com/gbolmier/funk-svd

  Cloning https://github.com/gbolmier/funk-svd to /tmp/pip-req-build-dbra7j_l
  Running command git clone -q https://github.com/gbolmier/funk-svd /tmp/pip-req-build-dbra7j_l
  Created wheel for funk-svd: filename=funk_svd-0.0.1.dev1-py3-none-any.whl size=9058 sha256=c67491206ac54c9251a8fb8f70f1fbe24d058e4d4b060c9470f976b9a42fbf68
  Stored in directory: /tmp/pip-ephem-wheel-cache-mkc4ou28/wheels/99/98/69/793c84ef2626b03661e3cddf49d8818cddbb694b0428adaeb4
Successfully built funk-svd


In [2]:
!pip install kaggle

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"olgamatykina","key":"255ba725b96dc08c7495d4e8132f7892"}'}

In [5]:
#Make a directory named kaggle and copy the kaggle.json file there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# change the permission of the file
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
import kaggle
!kaggle datasets download -d grouplens/movielens-20m-dataset

 90% 176M/195M [00:01<00:00, 152MB/s]
100% 195M/195M [00:01<00:00, 140MB/s]


In [7]:
from zipfile import ZipFile
file_name = 'movielens-20m-dataset.zip' #the file is your dataset exact name
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [8]:
from funk_svd import SVD

from sklearn.metrics import mean_absolute_error

import pandas as pd




In [9]:
df = pd.read_csv('rating.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [10]:
df.rename(columns={'userId': 'u_id'}, inplace=True)
df.rename(columns={'movieId': 'i_id'}, inplace=True)

In [11]:
train = df.sample(frac=0.8, random_state=7)
val = df.drop(train.index.tolist()).sample(frac=0.5, random_state=8)
test = df.drop(train.index.tolist()).drop(val.index.tolist())

svd = SVD(lr=0.001, reg=0.005, n_epochs=100, n_factors=15, early_stopping=True, shuffle=False, min_rating=1, max_rating=5)

svd.fit(X=train, X_val=val)

pred = svd.predict(test)
mae = mean_absolute_error(test['rating'], pred)

print(f'Test MAE: {mae:.2f}')

Preprocessing data...

Preprocessing data...

Epoch 1/100  | val_loss: 0.84 - val_rmse: 0.92 - val_mae: 0.71 - took 4.5 sec
Epoch 2/100  | val_loss: 0.80 - val_rmse: 0.90 - val_mae: 0.69 - took 3.5 sec
Epoch 3/100  | val_loss: 0.79 - val_rmse: 0.89 - val_mae: 0.68 - took 3.7 sec
Epoch 4/100  | val_loss: 0.78 - val_rmse: 0.88 - val_mae: 0.68 - took 3.2 sec
Epoch 5/100  | val_loss: 0.77 - val_rmse: 0.88 - val_mae: 0.68 - took 3.6 sec
Epoch 6/100  | val_loss: 0.76 - val_rmse: 0.87 - val_mae: 0.67 - took 3.8 sec
Epoch 7/100  | val_loss: 0.76 - val_rmse: 0.87 - val_mae: 0.67 - took 3.4 sec
Epoch 8/100  | val_loss: 0.76 - val_rmse: 0.87 - val_mae: 0.67 - took 3.4 sec
Epoch 9/100  | val_loss: 0.75 - val_rmse: 0.87 - val_mae: 0.67 - took 3.1 sec
Epoch 10/100 | val_loss: 0.75 - val_rmse: 0.87 - val_mae: 0.67 - took 3.0 sec
Epoch 11/100 | val_loss: 0.75 - val_rmse: 0.86 - val_mae: 0.66 - took 3.2 sec
Epoch 12/100 | val_loss: 0.75 - val_rmse: 0.86 - val_mae: 0.66 - took 3.2 sec
Epoch 13/100 | val

Коллаборативный + контентный подход

In [12]:
pip install lightfm

     |████████████████████████████████| 310 kB 4.6 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705376 sha256=8024040b6669ccc327a4a0d9c8abc194bf83d52991d37660f44a7cf992a241f3
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [13]:
import numpy as np

from lightfm.datasets import fetch_movielens

data = fetch_movielens(min_rating=5.0)

In [14]:
from lightfm import LightFM

In [15]:
model = LightFM(loss='warp')
model.fit(data['train'], epochs=30, num_threads=2)

In [16]:
from lightfm.evaluation import precision_at_k

In [17]:
print("Train precision: %.2f" % precision_at_k(model, data['train'], k=5).mean())
print("Test precision: %.2f" % precision_at_k(model, data['test'], k=5).mean())

Train precision: 0.41
Test precision: 0.06


In [18]:
def sample_recommendation(model, data, user_ids):

    n_users, n_items = data['train'].shape

    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]

        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)

sample_recommendation(model, data, [3, 25, 450])

User 3
     Known positives:
        Contact (1997)
        Air Force One (1997)
        In & Out (1997)
     Recommended:
        Lost Highway (1997)
        G.I. Jane (1997)
        Saint, The (1997)
User 25
     Known positives:
        Fargo (1996)
        Godfather, The (1972)
        L.A. Confidential (1997)
     Recommended:
        Fargo (1996)
        Godfather, The (1972)
        Titanic (1997)
User 450
     Known positives:
        Event Horizon (1997)
        Scream (1996)
        Conspiracy Theory (1997)
     Recommended:
        Scream (1996)
        Air Force One (1997)
        Titanic (1997)
